***GENERATED CODE FOR gradientboosting PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA FROM DATABRICKS FILESYSTEM.***

In [ ]:
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class DBFSConnector:

    def fetch(spark, config):
        df = spark.read.\
            options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                    inferschema='true',
                    delimiter=eval(config)["delimiter"])\
            .csv(eval(config)['url'])
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                                         meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ChargeDetailID0', 'transformation_label': 'String Indexer'}], 'feature': 'ChargeDetailID0', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '2020', 'mean': '', 'stddev': '', 'min': '008ecc10-5900-4280-b5d6-fceb6e8d1dde', 'max': 'ff54b805-3ca5-4d22-999e-f75de7d0b71d', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ChargeDetailID0'}, {'feature_label': 'ChargeDetailID0', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ChargeDetailID0')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'RVUModifier', 'transformation_label': 'String Indexer'}], 'feature': 'RVUModifier', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '2020', 'mean': '26.0', 'stddev': '0.0', 'min': '26', 'max': '26', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'RVUModifier'}, {'feature_label': 'RVUModifier', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('RVUModifier')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Gender', 'transformation_label': 'String Indexer'}], 'feature': 'Gender', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '2020', 'mean': '', 'stddev': '', 'min': 'F', 'max': 'M', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Gender'}, {'feature_label': 'Gender', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Gender')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PayerName', 'transformation_label': 'String Indexer'}], 'feature': 'PayerName', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '2020', 'mean': '', 'stddev': '', 'min': 'AARPMedicareComplete', 'max': 'WorkersComp', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PayerName'}, {'feature_label': 'PayerName', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PayerName')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PayerCategory', 'transformation_label': 'String Indexer'}], 'feature': 'PayerCategory', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '2020', 'mean': '', 'stddev': '', 'min': 'Aetna', 'max': 'WorkComp', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PayerCategory'}, {'feature_label': 'PayerCategory', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PayerCategory')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PayerGroup', 'transformation_label': 'String Indexer'}], 'feature': 'PayerGroup', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '2020', 'mean': '', 'stddev': '', 'min': 'Aetna', 'max': 'OtherGovernment', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PayerGroup'}, {'feature_label': 'PayerGroup', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PayerGroup')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'SourceKey58', 'transformation_label': 'String Indexer'}], 'feature': 'SourceKey58', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '2020', 'mean': '', 'stddev': '', 'min': 'CO109', 'max': 'PR51', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'SourceKey58'}, {'feature_label': 'SourceKey58', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('SourceKey58')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ReasonDate', 'transformation_label': 'String Indexer'}], 'feature': 'ReasonDate', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '2020', 'mean': '', 'stddev': '', 'min': '01/05/2020', 'max': '31/03/2020', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ReasonDate'}, {'feature_label': 'ReasonDate', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ReasonDate')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'FacilityID63', 'transformation_label': 'String Indexer'}], 'feature': 'FacilityID63', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '2020', 'mean': '', 'stddev': '', 'min': '087613d0-c7f3-4bcd-900a-abbc6ee3bdad', 'max': 'fde71f17-372e-4cde-95a7-d8b9b45a015a', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'FacilityID63'}, {'feature_label': 'FacilityID63', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('FacilityID63')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'DepartmentID', 'transformation_label': 'String Indexer'}], 'feature': 'DepartmentID', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '2020', 'mean': '', 'stddev': '', 'min': '0a414089-7b3a-4378-ba12-23a2d7272bcf', 'max': 'fb68b47d-bad4-42c2-9ad0-9816864ea30a', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'DepartmentID'}, {'feature_label': 'DepartmentID', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('DepartmentID')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split


def gradientboostingclassifier(df, labels, features):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = (sparkDF.toPandas())
    model_gbm_sklearn = GradientBoostingClassifier(**optuna_parameters)
    X_train, X_test, y_train, y_test = train_test_split(
        df[features], df[labels])
    model_gbm_sklearn.fit(X_train, y_train)
    display(" Accuracy of Model : %s" %
            model_gbm_sklearn.score(X_test, y_test))

    data = {'model': model_gbm_sklearn,
            'X_test': X_test,
            'y_test': y_test,
            'label': labels,
            'columnNames': df.columns}
    return data


***READING DATAFRAME***

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

#%run gradientboostingHooks.ipynb
try:
	#sourcePreExecutionHook()

	dbfs = DBFSConnector.fetch(spark, "{'url': '/Whitespace/whitespace_Classification_denails.csv', 'file_type': 'Delimeted', 'delimiter': ',', 'is_header': 'Use Header Line'}")
	#sourcePostExecutionHook(dbfs)

except Exception as ex: 
	logging.error(ex)


***TRANSFORMING DATAFRAME***

In [ ]:
#%run gradientboostingHooks.ipynb
try:
	#transformationPreExecutionHook()

	autofe = TransformationMain.run(dbfs,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "ChargeDetailID0", "transformation_label": "String Indexer"}], "feature": "ChargeDetailID0", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "2020", "mean": "", "stddev": "", "min": "008ecc10-5900-4280-b5d6-fceb6e8d1dde", "max": "ff54b805-3ca5-4d22-999e-f75de7d0b71d", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ChargeDetailID0"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Amount1", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "2020", "mean": "238.21", "stddev": "659.0", "min": "4.0", "max": "23900.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Amount1"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Amount2", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "2020", "mean": "84.76", "stddev": "193.44", "min": "-566.3", "max": "3368.38", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Amount2"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "CPTCodeID3", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "2020", "mean": "1742.97", "stddev": "1211.31", "min": "1", "max": "4079", "missing": "0"}, "updatedLabel": "CPTCodeID3"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "DiagnosisCodeID2", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "2020", "mean": "29585.15", "stddev": "22325.19", "min": "13501", "max": "94911", "missing": "0"}, "updatedLabel": "DiagnosisCodeID2"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "DiagnosisCodeID1", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "2020", "mean": "38130.89", "stddev": "20693.38", "min": "13571", "max": "94819", "missing": "0"}, "updatedLabel": "DiagnosisCodeID1"}, {"transformationsData": [{"feature_label": "RVUModifier", "transformation_label": "String Indexer"}], "feature": "RVUModifier", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "2020", "mean": "26.0", "stddev": "0.0", "min": "26", "max": "26", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "RVUModifier"}, {"transformationsData": [{"feature_label": "Gender", "transformation_label": "String Indexer"}], "feature": "Gender", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "2020", "mean": "", "stddev": "", "min": "F", "max": "M", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Gender"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "EnterpriseID18", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "2020", "mean": "1.0", "stddev": "0.07", "min": "1", "max": "2", "missing": "0"}, "updatedLabel": "EnterpriseID18"}, {"transformationsData": [{"feature_label": "PayerName", "transformation_label": "String Indexer"}], "feature": "PayerName", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "2020", "mean": "", "stddev": "", "min": "AARPMedicareComplete", "max": "WorkersComp", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PayerName"}, {"transformationsData": [{"feature_label": "PayerCategory", "transformation_label": "String Indexer"}], "feature": "PayerCategory", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "2020", "mean": "", "stddev": "", "min": "Aetna", "max": "WorkComp", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PayerCategory"}, {"transformationsData": [{"feature_label": "PayerGroup", "transformation_label": "String Indexer"}], "feature": "PayerGroup", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "2020", "mean": "", "stddev": "", "min": "Aetna", "max": "OtherGovernment", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PayerGroup"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "SpecialityID", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "2020", "mean": "36.22", "stddev": "30.45", "min": "-1", "max": "120", "missing": "0"}, "updatedLabel": "SpecialityID"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "TaxonomyID", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "2020", "mean": "353.68", "stddev": "168.38", "min": "-1", "max": "770", "missing": "0"}, "updatedLabel": "TaxonomyID"}, {"transformationsData": [{"feature_label": "SourceKey58", "transformation_label": "String Indexer"}], "feature": "SourceKey58", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "2020", "mean": "", "stddev": "", "min": "CO109", "max": "PR51", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "SourceKey58"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Amount59", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "2020", "mean": "238.21", "stddev": "659.0", "min": "4.0", "max": "23900.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Amount59"}, {"transformationsData": [{"feature_label": "ReasonDate", "transformation_label": "String Indexer"}], "feature": "ReasonDate", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "2020", "mean": "", "stddev": "", "min": "01/05/2020", "max": "31/03/2020", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ReasonDate"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PracticeID", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "2020", "mean": "1.0", "stddev": "0.07", "min": "1", "max": "2", "missing": "0"}, "updatedLabel": "PracticeID"}, {"transformationsData": [{"feature_label": "FacilityID63", "transformation_label": "String Indexer"}], "feature": "FacilityID63", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "2020", "mean": "", "stddev": "", "min": "087613d0-c7f3-4bcd-900a-abbc6ee3bdad", "max": "fde71f17-372e-4cde-95a7-d8b9b45a015a", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "FacilityID63"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "CPTCodeID66", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "2020", "mean": "1742.97", "stddev": "1211.31", "min": "1", "max": "4079", "missing": "0"}, "updatedLabel": "CPTCodeID66"}, {"transformationsData": [{"feature_label": "DepartmentID", "transformation_label": "String Indexer"}], "feature": "DepartmentID", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "2020", "mean": "", "stddev": "", "min": "0a414089-7b3a-4378-ba12-23a2d7272bcf", "max": "fb68b47d-bad4-42c2-9ad0-9816864ea30a", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "DepartmentID"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "status", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "2020", "mean": "0.49", "stddev": "0.5", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "status"}, {"feature": "ChargeDetailID0_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "2020", "mean": "489.24", "stddev": "400.55", "min": "0.0", "max": "1327.0", "missing": "0"}, "updatedLabel": "ChargeDetailID0_stringind..."}, {"feature": "RVUModifier_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "2020", "mean": "0.0", "stddev": "0.0", "min": "0", "max": "0", "missing": "0"}, "updatedLabel": "RVUModifier_stringindexer..."}, {"feature": "Gender_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "2020", "mean": "0.38", "stddev": "0.49", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Gender_stringindexer_tran..."}, {"feature": "PayerName_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "2020", "mean": "3.77", "stddev": "7.65", "min": "0.0", "max": "58.0", "missing": "0"}, "updatedLabel": "PayerName_stringindexer_t..."}, {"feature": "PayerCategory_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "2020", "mean": "1.96", "stddev": "3.0", "min": "0.0", "max": "19.0", "missing": "0"}, "updatedLabel": "PayerCategory_stringindex..."}, {"feature": "PayerGroup_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "2020", "mean": "0.84", "stddev": "1.61", "min": "0.0", "max": "7.0", "missing": "0"}, "updatedLabel": "PayerGroup_stringindexer_..."}, {"feature": "SourceKey58_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "2020", "mean": "6.66", "stddev": "12.36", "min": "0.0", "max": "86.0", "missing": "0"}, "updatedLabel": "SourceKey58_stringindexer..."}, {"feature": "ReasonDate_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "2020", "mean": "10.63", "stddev": "13.94", "min": "0.0", "max": "88.0", "missing": "0"}, "updatedLabel": "ReasonDate_stringindexer_..."}, {"feature": "FacilityID63_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "2020", "mean": "17.35", "stddev": "18.96", "min": "0.0", "max": "87.0", "missing": "0"}, "updatedLabel": "FacilityID63_stringindexe..."}, {"feature": "DepartmentID_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "2020", "mean": "1.98", "stddev": "3.3", "min": "0.0", "max": "27.0", "missing": "0"}, "updatedLabel": "DepartmentID_stringindexe..."}]}))

	#transformationPostExecutionHook(autofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run gradientboostingHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=GradientBoostingClassifier(autofe, ["ChargeDetailID0_stringindexer", "Amount1", "Amount2", "CPTCodeID3", "DiagnosisCodeID2", "DiagnosisCodeID1", "RVUModifier_stringindexer", "Gender_stringindexer", "EnterpriseID18", "PayerName_stringindexer", "PayerCategory_stringindexer", "PayerGroup_stringindexer", "SpecialityID", "TaxonomyID", "SourceKey58_stringindexer", "Amount59", "ReasonDate_stringindexer", "PracticeID", "FacilityID63_stringindexer", "CPTCodeID66", "DepartmentID_stringindexer"], "status")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)
